QNN - Quantum neural network - binary classification

In [1]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit_aer import Aer
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize

In [8]:
import pandas as pd
# Load dataset
df = pd.read_csv("ionosphere.data",header=None)
# First 34 columns → features
X = df.iloc[:, :-1].values
# Last column → labels ('g' or 'b')
y_raw = df.iloc[:, -1].values
# Convert labels to 0 / 1
y = np.array([1 if label == 'g' else 0 for label in y_raw])

2.MinMaxScaler - to scale the values to specific range here 0 to pi - bcz in quantum we want to use to ry - rotation.

In [9]:
scaler = MinMaxScaler(feature_range=(0, np.pi))
X_scaled = scaler.fit_transform(X)

3.Dimensionality reduction

In [10]:
pca = PCA(n_components=2)
X_reduced = pca.fit_transform(X_scaled)

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X_reduced, y, test_size=0.2, random_state=42)

In [16]:
def qnn_circuit(x, theta):
    qc = QuantumCircuit(2) # 2 qubit
    qc.ry(x[0], 0)  #encode data
    qc.ry(x[1], 1)
    qc.ry(theta[0], 0) #tainable_parms
    qc.ry(theta[1], 1)
    qc.cx(0, 1) #entanglement
    qc.measure_all() #measurment
    return qc


In [12]:
backend = Aer.get_backend("qasm_simulator")

def predict(x, theta, shots=512):
    qc = qnn_circuit(x, theta)
    result = backend.run(qc, shots=shots).result()
    counts = result.get_counts()

    # Class 1 if qubit 0 = 1
    p1 = (counts.get('10', 0) + counts.get('11', 0)) / shots
    return p1

In [14]:
def loss(theta):
    total_loss = 0
    for i in range(len(X_train)):
        p = predict(X_train[i], theta)
        total_loss += (p - y_train[i])**2
    return total_loss / len(X_train)

In [17]:
theta_init = np.random.rand(2)
"""What is COBYLA?

Constrained

Optimization

BY

Linear

Approximation

Why COBYLA is used in QML?Works without gradients,Robust to noise,Standard for Qiskit & QNNs
->Gradient-based methods struggle with noisy quantum outputs."""

result = minimize(
    loss,
    theta_init,
    method="COBYLA",
    options={"maxiter": 50}
)

theta_opt = result.x
print("Optimized parameters:", theta_opt)

Optimized parameters: [0.52404424 1.06610068]


In [18]:
correct = 0

for i in range(len(X_test)):
    p = predict(X_test[i], theta_opt)
    y_pred = 1 if p >= 0.5 else 0

    if y_pred == y_test[i]:
        correct += 1

accuracy = correct / len(X_test)
print("Test Accuracy:", accuracy)

Test Accuracy: 0.5915492957746479


In [21]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Train Logistic Regression
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Predict
y_pred_lr = clf.predict(X_test)

# Accuracy
acc_lr = accuracy_score(y_test, y_pred_lr)
print("Logistic Regression Accuracy:", acc_lr)


Logistic Regression Accuracy: 0.4788732394366197


In [23]:
from qiskit import QuantumCircuit, transpile
from qiskit_aer import AerSimulator
import numpy as np
backend = AerSimulator()


backend = Aer.get_backend("qasm_simulator")

def quantum_circuit(x, theta):
    qc = QuantumCircuit(2, 2)

    # Data encoding
    qc.ry(x[0], 0)
    qc.ry(x[1], 1)

    # Layer 1
    qc.ry(theta[0], 0)
    qc.ry(theta[1], 1)
    qc.cx(0, 1)

    # Layer 2 (NEW)
    qc.ry(theta[2], 0)
    qc.ry(theta[3], 1)
    qc.cx(1, 0)

    qc.measure([0, 1], [0, 1])
    return qc


def predict(x, theta, shots=512):
    qc = quantum_circuit(x, theta)

    tqc = transpile(qc, backend)
    job = backend.run(tqc, shots=shots)
    result = job.result()
    counts = result.get_counts()

    p1 = 0
    for bitstring, count in counts.items():
        if bitstring[0] == '1':   # measure qubit-0
            p1 += count

    return p1 / shots

def loss(theta):
    total_loss = 0
    for i in range(len(X_train)):
        p = predict(X_train[i], theta)
        y = y_train[i]
        total_loss += -(y*np.log(p+1e-6) + (1-y)*np.log(1-p+1e-6))
    return total_loss / len(X_train)
from scipy.optimize import minimize

theta_init = np.random.rand(4)

result = minimize(
    loss,
    theta_init,
    method="COBYLA",
    options={"maxiter": 150}
)

theta_opt = result.x
print("Optimized parameters:", theta_opt)
correct = 0

for i in range(len(X_test)):
    p = predict(X_test[i], theta_opt)
    y_pred = 1 if p >= 0.5 else 0
    if y_pred == y_test[i]:
        correct += 1

accuracy = correct / len(X_test)
print("Deeper QNN Accuracy:", accuracy)



Optimized parameters: [1.17279109 1.34176244 0.39390673 0.0487656 ]
Deeper QNN Accuracy: 0.704225352112676


In [24]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Train Logistic Regression
clf = LogisticRegression()
clf.fit(X_train, y_train)

# Predict
y_pred_lr = clf.predict(X_test)

# Accuracy
acc_lr = accuracy_score(y_test, y_pred_lr)
print("Logistic Regression Accuracy:", acc_lr)

Logistic Regression Accuracy: 0.4788732394366197
